# PA4 - String Kernals 

Zhanchong Deng  
A15491777

### Necessary Imports

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import kernel as pa4

Loading Data in

In [2]:
training = pa4.loadData('data/pa4train.txt')
testing = pa4.loadData('data/pa4test.txt')

Parse them into DataFrames

In [3]:
training = np.array(pd.DataFrame(training).astype({0:str, 1:int}))
testing = np.array(pd.DataFrame(testing).astype({0:str, 1:int}))

___

### Disclaimer: only train if there is something wrong with original algorithm. Training takes very long  

Save each p in **saved/delta_idx_.csv** to avoid long training time.

In [38]:
for length in [3,4,5]:
    p = pa4.Perceptron()
    p.fit(training, length)
    pd.DataFrame(p.delta, columns=['x','y']).to_csv("saved/delta"+ str(length) + ".csv")

Half way done
All done
Half way done
All done


___

### 1) Training/Testing Error for each p with k(s,t) = #occurances in s * #occurances in t

In [44]:
for length in [3,4,5]:
    p.delta = np.array(pd.read_csv(filepath_or_buffer="saved/delta"+ str(length) + ".csv", usecols=['x','y']))
    p.p = length
    print("Training Error for (p =", length, ")is:", p.error(training))
    print("Testing Error for (p=", length, ")is:", p.error(testing))

Training Error for (p=3 )is: 0.011846
Testing Error for (p=3 )is: 0.052770
Training Error for (p=4 )is: 0.007163
Testing Error for (p=3 )is: 0.027704
Training Error for (p=5 )is: 0.003581
Testing Error for (p=5 )is: 0.030343


___

Now train with second kernel function.

In [7]:
for length in [3,4,5]:
    p2 = pa4.Perceptron()
    p2.fit(training, length)
    pd.DataFrame(p2.delta_idx, columns=['delta_idx']).to_csv("saved2/delta_idx"+ str(length) + ".csv")

Half way done
Half way done
Half way done


___

### 2) Training/Testing Error for each p with k(s,t) = #common words only

In [45]:
for length in [3,4,5]:
    p2.delta_idx = pd.read_csv("saved2/delta_idx"+ str(length) + ".csv")['delta_idx'].to_list()
    p2.p = length
    print("Training Error for (p =", length, ")is:", p2.error(training))
    print("Testing Error for (p=", length, ")is:", p2.error(testing))

Training Error for (p=3 )is: 0.012121
Testing Error for (p=3 )is: 0.040897
Training Error for (p=4 )is: 0.006612
Testing Error for (p=3 )is: 0.025066
Training Error for (p=5 )is: 0.003581
Testing Error for (p=5 )is: 0.030343


___

Extracting p=5 info

In [4]:
df = pd.read_csv(filepath_or_buffer="saved/delta5.csv", usecols=['x','y'])
positives = np.array(df['x'][df['y']==1])
negatives = np.array(df['x'][df['y']==-1])

Record occurances of all substrings in w

In [5]:
sub_strings = {}
# Add positives
for string in positives:
    for i in range(len(string) - 5 + 1):
        if string[i:i+5] in sub_strings.keys():
            sub_strings[string[i:i+5]] += 1
        else:
            sub_strings[string[i:i+5]] = 1
# Subtract negatives
for string in negatives:
    for i in range(len(string) - 5 + 1):
        if string[i:i+5] in sub_strings.keys():
            sub_strings[string[i:i+5]] -= 1
        else:
            sub_strings[string[i:i+5]] = -1

In [6]:
counts = pd.DataFrame({'sub_string':pd.Series(list(sub_strings.keys())), 'count':pd.Series(list(sub_strings.values()))})

### 3) Intepreting Classifier p = 5

The two most positive sub_strings are:

In [14]:
counts.sort_values(by='count', ascending = False)[:2]

,sub_string,count
60,WDTAG,3
61,DTAGQ,3


___

### 4) Code from Kernel.py

```python

'''
This file contains all methods/class working with string kernels.
'''
import numpy as np
import pandas as pd

def loadData(fp):
    newfile = open(fp, 'r')
    newfile.seek(0)
    raw_strings = newfile.read().split("\n")[:-1]
    return np.array([np.array(entry.split(" "), dtype="str") for entry in raw_strings])

'''
Calculates common substring of length p in both s and t. If a string occurs a times in s
and b times in t, then increment total count by a * b.
@param p is the length of substring
@param s is the first string.
@param t is the econd string.
'''
def kernel(s, t, p):
    product = 0
    # Maps every substring to a number within s and t of length p
    map_s = generate_map(s, p)
    map_t = generate_map(t, p)
    # Iterate through one of them
    for key_s in [*map_s]:
        if key_s in [*map_t]:
            product += map_s[key_s] * map_t[key_s]
    return product

def generate_map(string, p):
    uniques = {}
    for i in range(len(string) - p + 1):
        if string[i:i+p] not in uniques.keys():
            uniques[string[i:i+p]] = 1
        else:
            uniques[string[i:i+p]] += 1
    return uniques

def kernel2(s, t, p):
    common = 0
    # Record uniques
    unique_s = []
    unique_t = []
    for i in range(len(s) - p + 1):
        unique_s.append(s[i:i+p])
    for j in range(len(t) - p + 1):
        unique_t.append(t[i:i+p])
    unique_s = set(unique_s)
    unique_t = set(unique_t)
    for key_s in unique_s:
        if key_s in unique_t:
            common += 1
    return common


class Perceptron():
    # Instead of w, keep track of index of x, y that made errors
    delta = []
    # store p as a instance
    p = -1
    
    def fit(self, training_data, p):
        self.p = p
        # One pass only
        count = 0
        switch = True
        for i in range(len(training_data)):
            # progress report
            if count > len(training_data)/2 and switch:
                print("Half way done")
                switch = False
            count += 1
            data = training_data[i]
            # Edge case:
            if len(self.delta) == 0:
                self.delta.append(data)
                continue
            # Update case:
            if data[1] * self.w_dot_phi(data) <= 0:
                self.delta.append(data)
        print("All done")
            
    '''
    Helper method which calculates <w_t, phi(x)>
    '''
    def w_dot_phi(self, x):
        # Iterate through all x in list, compute sum of kernals
        dot_product = 0
        for error_data in self.delta:
            dot_product += error_data[1] * kernel(error_data[0], x[0], self.p)
        return dot_product
    
    '''
    Functions for Prediction/Testing
    '''
    # Calculate Error
    def error(self, testing_data):
        # Depending on what method is, calculate predictions
        predictions = self.predict_pass(testing_data)
        return np.mean(predictions != testing_data[:,1])
    
    def predict_pass_one(self, a_test_data):
        if self.w_dot_phi(a_test_data) >= 0:
            return 1
        else:
            return -1
    
    def predict_pass(self, testing_data):
        return np.apply_along_axis(self.predict_pass_one, 1, testing_data[:,:-1])

```